In [1]:
# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
get_ipython().magic('env CUDA_VISIBLE_DEVICES =  ')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp      # will come in handy due to the size of the data
import os.path
import random
import time
import scipy
from collections import OrderedDict
import io
from datetime import datetime
import gc # garbage collector
import sklearn
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
import math
import sys
from collections import defaultdict
import re
import logging
from sklearn.model_selection import KFold
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
get_ipython().magic('matplotlib inline')
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')

env: CUDA_VISIBLE_DEVICES=


/home/kapok/pyenv35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/kapok/pyenv35/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
RAW_INPUT_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/raw_input.csv'
MEDIAN_INPUT_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/median_input.csv'
MEAN_INPUT_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/Resnet_pytorch_state_438453_2017-12-14_06_43_50.pth'


In [3]:
# raw_pred = pd.read_csv(MEDIAN_INPUT_PATH)#, names = ["_id", "category_id"])

In [4]:
# raw_pred.iloc[[1767852],[0]] = 23597750
# print(raw_pred['_id'][1767852])
# raw_pred.to_csv('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/result_ok.csv',index=False)

In [5]:
raw_pred = pd.read_csv(RAW_INPUT_PATH)#[-1000:]#, names = ["_id", "category_id"]
median_pred = pd.read_csv(MEDIAN_INPUT_PATH)

In [6]:
raw_pred.columns = ['_id', 'category_id']

In [7]:
mode_pred = raw_pred.groupby(['_id'])['category_id'].agg(lambda x: scipy.stats.mode(x)[0])
mode_count = raw_pred.groupby(['_id'])['category_id'].agg(lambda x: scipy.stats.mode(x)[1])

In [8]:
mode_pred = mode_pred.reset_index(drop=False)
mode_count = mode_count.reset_index(drop=False)
mode_count.columns = ['_id', 'mode_count']
mode_pred['mode_count'] = mode_count['mode_count']

In [ ]:
bal = median_pred[1769675:]
median_pred = median_pred[:1769675]
mode_pred = mode_pred[:1768182]

In [17]:
median_pred__ = pd.merge(median_pred, mode_pred[['_id', 'category_id']], how='left', left_on=['_id'], right_on = ['_id'])
median_pred__ = pd.merge(median_pred__, mode_count[['_id', 'mode_count']], how='left', left_on=['_id'], right_on = ['_id'])


In [21]:
median_pred['mask'] = (median_pred__['category_id_x'] != median_pred__['category_id_y'])&(median_pred__['mode_count']>1)
#median_pred['mask'] = (median_pred['category_id']!=mode_pred['category_id'])&(mode_pred['mode_count']>1)
#median_pred['mask'] = (median_pred['_id']!=mode_pred['_id'])
median_pred['another'] = median_pred__['category_id_y']
median_pred['mask'] = median_pred['mask'].astype(int)

In [12]:
(raw_pred['_id']==14).sum()

1

1678604

In [13]:
raw_pred['_id'].value_counts()

12225710    4
17788793    4
13566722    4
8452861     4
21224       4
8934        4
4293330     4
21970803    4
4287183     4
80581       4
195261      4
9470473     4
8565428     4
17344384    4
4767619     4
21140144    4
18038364    4
8974217     4
16915105    4
4330144     4
982859      4
17802006    4
299632      4
17887751    4
204535      4
4421370     4
17497021    4
9213702     4
17449898    4
9165721     4
           ..
18687458    1
17732178    1
21922384    1
20777396    1
13609357    1
918080      1
5110335     1
13492796    1
17676855    1
13478453    1
18365387    1
9273904     1
1924589     1
6141424     1
18084350    1
21828130    1
21819934    1
17617434    1
17613336    1
22533969    1
22916595    1
22910452    1
10345981    1
813581      1
8533504     1
12729857    1
142851      1
4993542     1
13378052    1
4098        1
Name: _id, Length: 1769675, dtype: int64

In [ ]:
print(len(median_pred['category_id']),len(mode_pred['category_id']))

In [22]:
median_pred['real_pred'] = median_pred['category_id']*(1-median_pred['mask']) + median_pred['another']*median_pred['mask']

In [23]:
to_save = median_pred[['_id', 'real_pred']]
to_save.columns = ['_id', 'category_id']

In [ ]:
to_save.to_csv('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/sub.csv', index=False, sep=',')

In [ ]:
(median_pred['category_id']!=mode_pred['category_id'])

In [ ]:
median_pred = median_pred[1:]
median_pred = median_pred.reset_index(drop=True)